# 모델 매개변수 최적화하기
데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트를 해보자.

각 반복 단계에서 모델은 출력을 추측 -> 추측과 정답 사이의 오류(손실 loss)를 계산 -> 매개변수에 대한 오류의 도함수(derivative)를 수집 -> 경사하강법을 사용해 파라미터 **최적화(optimize)**

## 기본(Pre-requisite) 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라매터(Hyperparameter)
하이퍼파라매터는 모델 최적화 과정에서 제어할 수 있는 조절 가능한 매개변수이다.

* 에폭(epoch) 수 - 데이터셋을 반복하는 횟수
* 배치 크기(batch size) - 각 에폭에서 한 번에 모델에 전달할 데이터 샘플의 수
* 학습율(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 단계(Optimization Loop)
하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화한다. 최적화 단계의 각 반복(iteration)을 에폭이라고 부른다.

* 학습 단계(train loop) - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴
* 검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)

### 손실 함수(loss function)
손실 함수(loss function)는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정, 학습 중 이 값을 최소화하려고 한다. |예측값 - 정답(label)| = 손실(loss)
* 회귀 문제(regression task)에 사용하는 ```nn.MSELoss```(평균 제곱 오차(MSE; Mean Square Error))
* 분류(classification)에 사용하는 ```nn.NLLLoss```(음의 로그 우도(Negative Log Likelihood))
* 이 외 ```nn.LogSoftmax```, ```nn.NLLLoss```를 합친 ```nn.CrossEntropyLoss```

모델의 출력 로짓(logit)을 ```nn.CrossEntropyLoss```에 전달하여 로짓(logit)을 정규화하고 예측 오류 계산

In [4]:
# 손실 함수 초기화
loss_fn = nn.CrossEntropyLoss()

### 옵티마이저(Optimizer)
* 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
* 최적화 알고리즘은 이 과정이 수행되는 방식 정의
* 모든 최적화 절차(logic)는 ```optimizer``` 객체에 캡슐화(encapsulate)
* 여기서는 SGD 옵티마이저 사용, ADAM이나 RMSProp과 같은 다양한 옵티마이저 존재
---
* 학습하려는 모델의 매개변수와 학습율(learning rate) 하이퍼파라매터를 등록하여 옵티마이저를 초기화

```optimizer = torch.optim.SGD(model.parameters(), lr=learniing_rate)```

#### 학습 단계(loop)에서의 최적화 세단계
1. ```optimizer.zero_grad()```를 호출하여 모델 매개변수의 변화도 재설정, 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
2, ```loss.backward()```를 호출해 예측 손실(prediction loss)을 역전파, PyTorch는 각 매개변수에 대한 손실의 변화도 저장
3. 변화도를 계산한 뒤에는 ```optimizer.step()```을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

## 전체 구현
최적화 코드를 반복하여 수행하는 *train_loop*와 테스트 데이터로 모델의 성능을 측정하는 *test_loop*를 정의

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
# 손실 함수와 옵티마이저 초기화 후 전달
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.244654  [    0/60000]
loss: 2.238784  [ 6400/60000]
loss: 2.230104  [12800/60000]
loss: 2.269010  [19200/60000]
loss: 2.206764  [25600/60000]
loss: 2.213145  [32000/60000]
loss: 2.230662  [38400/60000]
loss: 2.195183  [44800/60000]
loss: 2.203794  [51200/60000]
loss: 2.238241  [57600/60000]
Test Error: 
 Accuracy: 30.8%, Avg loss: 0.034537 

Epoch 2
-------------------------------
loss: 2.158077  [    0/60000]
loss: 2.149432  [ 6400/60000]
loss: 2.133123  [12800/60000]
loss: 2.227283  [19200/60000]
loss: 2.089399  [25600/60000]
loss: 2.115180  [32000/60000]
loss: 2.157061  [38400/60000]
loss: 2.090354  [44800/60000]
loss: 2.124865  [51200/60000]
loss: 2.182940  [57600/60000]
Test Error: 
 Accuracy: 32.2%, Avg loss: 0.033225 

Epoch 3
-------------------------------
loss: 2.043435  [    0/60000]
loss: 2.031460  [ 6400/60000]
loss: 2.013500  [12800/60000]
loss: 2.180266  [19200/60000]
loss: 1.954692  [25600/60000]
loss: 2.010974  [32000/600